# IA de Detecção de Tuberculose 
 Detecção de Tuberculose em pacientes através da análise de Raio X

## 1.Importação de Módulos

In [5]:
# Importação de módulos
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras import layers
import os
import kagglehub

## 2.Base de Dados

In [6]:
# Caminho da pasta do usuário
home_dir = os.path.expanduser("~")

# Caminho onde os datasets do kagglehub são armazenados
dataset_name = "raddar/tuberculosis-chest-xrays-montgomery"
dataset_folder = dataset_name.replace("/", os.sep)  
dataset_path = os.path.join(home_dir, ".cache", "kagglehub", "datasets", dataset_folder, "versions", "1")

# Verifica se o dataset já foi instalado
if not os.path.exists(dataset_path) or not os.listdir(dataset_path):
    print("Baixando o dataset...")
    path = kagglehub.dataset_download("raddar/tuberculosis-chest-xrays-montgomery")
    print("Download concluído! Arquivos em:", path)
else:
    print("Dataset já instalado. Usando arquivos locais em:", dataset_path)

Dataset já instalado. Usando arquivos locais em: C:\Users\ferna\.cache\kagglehub\datasets\raddar\tuberculosis-chest-xrays-montgomery\versions\1


## 3.Criando IA Convolucional

##### Separando casos normais e casos de tuberculose

In [ ]:
# Lendo arquivo compilado com os resultados médicos
exams_results = [f for f in os.listdir(dataset_path) if f.endswith('.csv')]
exams_result_path = os.path.join(dataset_path, exams_results[0])

images = os.path.join(dataset_path, 'images', 'images')

In [ ]:
# Criando DataFrame
df_exams_results = pd.read_csv(exams_result_path)
df_exams_results

,study_id,age,gender,findings
0,MCUCXR_0001_0.png,27,Female,normal
1,MCUCXR_0002_0.png,40,Female,normal
2,MCUCXR_0003_0.png,21,Female,normal
3,MCUCXR_0004_0.png,11,Female,normal
4,MCUCXR_0005_0.png,33,Male,normal
...,...,...,...,...
133,MCUCXR_0383_1.png,14,Female,calcified granuloma LUL behind 4th rib anterior
134,MCUCXR_0387_1.png,42,Male,inactive scars RUL
135,MCUCXR_0390_1.png,52,Male,Rt apical scarring. LUL scars and some infiltr...
136,MCUCXR_0393_1.png,30,Female,left pleural effusion; best seen on lateral vi...


In [25]:
# Tratando dados
df_exams_results['findings'] = df_exams_results['findings'].str.strip().str.upper()
df_exams_results

,study_id,age,gender,findings
0,MCUCXR_0001_0.png,27,Female,NORMAL
1,MCUCXR_0002_0.png,40,Female,NORMAL
2,MCUCXR_0003_0.png,21,Female,NORMAL
3,MCUCXR_0004_0.png,11,Female,NORMAL
4,MCUCXR_0005_0.png,33,Male,NORMAL
...,...,...,...,...
133,MCUCXR_0383_1.png,14,Female,CALCIFIED GRANULOMA LUL BEHIND 4TH RIB ANTERIOR
134,MCUCXR_0387_1.png,42,Male,INACTIVE SCARS RUL
135,MCUCXR_0390_1.png,52,Male,RT APICAL SCARRING. LUL SCARS AND SOME INFILTR...
136,MCUCXR_0393_1.png,30,Female,LEFT PLEURAL EFFUSION; BEST SEEN ON LATERAL VI...


In [ ]:
# Separando imagens normais e patológicas em duas listas
list_normal = []
list_tuberculosis = []

for i, row in df_exams_results.iterrows():
    name_image = row['study_id']
    if row['findings'] == 'NORMAL':
        list_normal.append(name_image)
    else:
        list_tuberculosis.append(name_image)


In [36]:
# Criando novos diretórios para casos normais e casos patológicos
dir_normal = 'Normal'
dir_tuberculosis = 'Tuberculosis'

path_normal = os.path.join(dataset_path, dir_normal)
path_tuberculosis = os.path.join(dataset_path, dir_tuberculosis)

os.makedirs(path_normal, exist_ok=True)
os.makedirs(path_tuberculosis, exist_ok=True)

In [ ]:
# Movendo imagens normais e patológicas para seus respectivos diretórios


## 4.Teste IA

In [17]:
# Exibindo resultados médicos das imagens fornecidas 
df_exams_results = pd.read_csv(exams_result_path)
df_exams_results

,study_id,age,gender,findings
0,MCUCXR_0001_0.png,27,Female,normal
1,MCUCXR_0002_0.png,40,Female,normal
2,MCUCXR_0003_0.png,21,Female,normal
3,MCUCXR_0004_0.png,11,Female,normal
4,MCUCXR_0005_0.png,33,Male,normal
...,...,...,...,...
133,MCUCXR_0383_1.png,14,Female,calcified granuloma LUL behind 4th rib anterior
134,MCUCXR_0387_1.png,42,Male,inactive scars RUL
135,MCUCXR_0390_1.png,52,Male,Rt apical scarring. LUL scars and some infiltr...
136,MCUCXR_0393_1.png,30,Female,left pleural effusion; best seen on lateral vi...
